In [1]:
SQL_SOURCE = 'https://vjs306.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

from IPython.display import Image

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-b43bed512ecf>, line 25)

In [ ]:
sQuery = '''
SELECT *
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery)), sep=',')

Table 1: Original dataset for citibike data

### Query #1

In [ ]:
sQuery1 = '''
SELECT * FROM citibike
WHERE tripduration<10800
ORDER BY tripduration DESC
LIMIT 3 
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery1)), sep=',')

Table 2: First 3 instances where trip duration is less than 3 hours

### Query #2

In [ ]:
sQuery2 = '''
SELECT AVG(tripduration) as avt, AVG(birth_year) as avb
FROM citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery2)), sep=',')

Table 3: Average trip duration and average year of birth

### Query #3

In [ ]:
sQuery3 = '''
SELECT *
FROM citibike
WHERE starttime >= '2015-02-02 00:00'
  AND starttime < '2015-02-03 00:00'
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery3)), sep=',')

Table 4: Citibike data for only February 2nd, 2015

### Query #4

In [ ]:
sQuery4 = '''
SELECT AVG(tripduration)
FROM citibike
WHERE extract(DOW from starttime) NOT IN (0,6) 
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery4)), sep=',')

Table 5: Average trip duration for weekdays, in seconds  (~11min)

### Query #5

In [ ]:
sQuery5 = '''
SELECT start_station_latitude, start_station_longitude
FROM citibike
GROUP BY start_station_latitude, start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery5)), sep=',')

Table 6: Showing the unique start stations, achieved by using groupby

### Query #6

In [ ]:
sQuery6 = '''
SELECT start_station_id, COUNT(*)
FROM citibike
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery6)), sep=',')

Table 7: Total count of number of trips originating at each station

### Query #7

In [ ]:
sQuery7 = '''
SELECT CDB_LatLng(start_station_latitude, start_station_longitude) as the_geom
FROM citibike
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery7)), sep=',')

Table 8: Creating coordinates, will be useful on next exercise

### Query #8

In [ ]:
sQuery8 = '''
SELECT CDB_TransformToWebmercator(
           CDB_LatLng(start_station_latitude, 
                      start_station_longitude)) as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id
FROM citibike
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery8)), sep=',')

Table 9: Interpreting the coordinates using the geom_webmercator

In [ ]:
Image("https://raw.githubusercontent.com/valschi/PUI2017_vjs306/master/HW7_vjs306/all_stations.png", width=450, height=450)

Figure1: Map corresponding to Query #8, plotting the coordinates of all the unique originating stations

### Query #9

In [ ]:
sQuery9 = '''
SELECT CDB_TransformToWebmercator(
           CDB_LatLng(start_station_latitude, 
                      start_station_longitude)) as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id 
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, 
                            start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,
                 500)
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery9)), sep=',')

Table 10: Coordinates for the stations within 500 meters from Times Square

In [ ]:
Image("https://raw.githubusercontent.com/valschi/PUI2017_vjs306/master/HW7_vjs306/stations%20times%20square.png", height = 450, width = 450)

Figure 2: Map corresponding to Query #9, plotting the stations within 500m of Times Square

### Query #10

In [ ]:
sQuery10 = '''
SELECT CDB_TransformToWebmercator(
           CDB_LatLng(start_station_latitude, 
                      start_station_longitude)) as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id, 
       AVG(tripduration) as ta
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, 
                            start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,
                 500)
GROUP BY start_station_id, 
start_station_latitude, start_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery10)), sep=',')

Table 11: adding Average trip duration as ta to be able to add weight to the dots

### Query #11

In [ ]:
sQuery11 = '''
SELECT ST_Makeline(
       CDB_TransformToWebmercator(
           CDB_LatLng(start_station_latitude, 
                      start_station_longitude)),
       CDB_TransformToWebmercator(
           CDB_LatLng(end_station_latitude, 
                      end_station_longitude))
       ) as the_geom_webmercator,          
       MIN(cartodb_id) as cartodb_id 
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, 
                            start_station_longitude)::geography,
                 CDB_LatLng(40.7577, -73.9857)::geography,
                 500)
GROUP BY start_station_id, start_station_latitude, start_station_longitude,
         end_station_id, end_station_latitude, end_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(sQuery11)), sep=',')

Table 12: This will allow plot lines between the stations

In [ ]:
Image("https://raw.githubusercontent.com/valschi/PUI2017_vjs306/master/HW7_vjs306/bike%20lines.png", height = 450, width = 450)

Figure 3: Map corresponding to Query #11, drawing lines between stations

## ASH Score 100. Extra Credit 0